Imports

In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import numpy as np

Training Shallow Model

In [ ]:
#read in snli json (can download from: https://nlp.stanford.edu/projects/snli/)
snli = pd.read_json("snli_1.0_train.jsonl", lines=True)
#change columns and columns values to what huggingface expects
snli = snli.rename(columns={"sentence1": "premise", "sentence2": "hypothesis", "gold_label":"label"})
snli['label'] = snli['label'].map({'entailment': int(0), 'neutral': int(1), 'contradiction': int(2)})
snli = snli.dropna()
snli['label'] = snli['label'].astype('int')
#randomly sample observations for training and evaluation sets
sample_size = 500
snli = snli.sample(frac=1)
snli_bias = snli.iloc[0:sample_size, :]
snli_bias_eval = snli.iloc[sample_size:, :]
#save training and evaluation sets
snli_bias.to_json("snli_bias_train.json", orient='records')
snli_bias_eval.to_json("snli_bias_eval.json", orient='records')

In [ ]:
#train model on small fraction of total training data
#for the ELECTRA-small model and SNLI dataset I used 500 observations + 20 epochs
#for a different model/dataset you will need to change number of observations or number of epochs
#these are changed to reach desired accuracy and 'certainty' (see next few code blocks)
#this step can take a lot of trial and error to get right
!python run.py --do_train --task nli --dataset snli_bias_train.json --output_dir ./trained_model_bias_500_20/ --per_device_train_batch_size 256 --num_train_epochs 20.0

In [ ]:
#evaluate on the rest of training data (this can be very slow, to save time first evaluate on much smaller subset of data)
#goal here is to achieve 60-70% accuracy and assign most predictions with probability > 0.9
!python run.py --do_eval --task nli --dataset snli_bias_eval.json --model ./content/trained_model_bias_500_20/ --output_dir ./eval_output_bias_500_20/

In [ ]:
#checking 'certainty' (portion of predictions with probability > 0.9)
#collecting probability of each model output
snli_eval = pd.read_json("./eval_output_bias_500_20/eval_predictions.jsonl", lines=True)
softmax = nn.Softmax()
l = snli_eval.shape[0]
max_prob = np.zeros((l))
for i in range(0, l):
    logits = snli_eval.predicted_scores[i]
    logits = torch.tensor(logits)
    probs = softmax(logits)
    probs = probs.numpy()
    max = np.max(probs)
    max_prob[i] = max
max_prob = pd.Series(max_prob)
#plot histogram of probabilities
max_prob.hist()
#ideally most (>50%) of predictions should have probability > 0.9

Training using reweighted loss function

In [ ]:
#key part here is to pass eval_predictions.json in as the dataset
#this file contains all the usual training data + predictions of shallow model
!python run_custom.py --do_train --task nli --dataset eval_predictions.jsonl --output_dir ./trained_model_reweighted/ --per_device_train_batch_size 256 --num_train_epochs 3.0
#to use annealing reweighted loss add '--annealing True'
#annealing can help regain evaluation performance on biased datasets

Model Evaluation

In [ ]:
#evaluating on snli eval. set
!python run.py --do_eval --task nli --dataset snli --model ./trained_model_reweighted --output_dir ./eval_snli_reweighted
#performace will likely be slightly worse than a model that has not been reweighted as it can no longer exploit artifacts to make easy predictions
#However, performance should be improved on a challenge dataset (e.g. HANS https://github.com/tommccoy1/hans)